# Clean dim_customers từ Silver lên Gold

## Mục tiêu
- Clean và transform dữ liệu từ Silver.dim_customers → Gold.dim_customers
- Đảm bảo data quality standards cho EDA
- Chuẩn bị dữ liệu sạch cho phân tích khách hàng

## Quy trình 6 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Data Quality Check** - Phân tích chất lượng hiện tại
3. **Business Rules** - Áp dụng business logic cleaning
4. **Data Cleaning** - Làm sạch và chuẩn hóa
5. **Load to Gold** - Load vào Gold database
6. **Validation** - Kiểm tra kết quả cuối cùng


## Bước 1: Import và Setup

### Mục tiêu
- Import các thư viện cần thiết
- Load biến môi trường từ file .env
- Tạo kết nối tới Silver và Gold database
- Thiết lập môi trường làm việc


In [14]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:00:50


## Bước 2: Load dữ liệu từ Silver

### Mục tiêu
- Load toàn bộ bảng dim_customers từ Silver database
- Hiển thị thông tin cơ bản về dataset
- Kiểm tra cấu trúc dữ liệu hiện tại


In [15]:
# Load dữ liệu dim_customers từ Silver database
print("=== LOADING DIM_CUSTOMERS FROM SILVER ===")

# Load toàn bộ bảng dim_customers
customers_df = pd.read_sql("SELECT * FROM dim_customers", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {customers_df.shape}")
print(f"Columns: {list(customers_df.columns)}")
print(f"Memory usage: {customers_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(customers_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(customers_df.dtypes)


=== LOADING DIM_CUSTOMERS FROM SILVER ===
Shape: (36090, 27)
Columns: ['customer_id', 'id', 'name', 'shop_id', 'gender', 'phone', 'order_count', 'succeed_order_count', 'returned_order_count', 'purchased_amount', 'last_order_at', 'inserted_at', 'updated_at', 'is_block', 'is_discount_by_level', 'active_levera_pay', 'reward_point', 'current_debts', 'count_referrals', 'referral_code', 'fb_id', 'order_sources', 'conversation_link', 'address_province_id', 'address_district_id', 'address_commune_id', 'address_full']
Memory usage: 38.06 MB

=== SAMPLE DATA ===
                            customer_id                                    id  \
0  c7a583d6-392d-4308-a02b-67c001ee03db  e906c5c7-ea19-42e5-a971-57cf09c94417   
1  590ba7de-e4fe-43b3-8013-27fc69937edf  6c5b40d2-ae42-4e13-8019-f3f9127d75a0   
2  a9e62388-57c2-4834-a87c-7c1025aea92d  765c531b-743f-4f30-b6b0-0d6f3579630f   

          name    shop_id gender       phone  order_count  \
0    Thinh Bui  230361475      M  0903693389           

## Bước 3: Data Quality Check

### Mục tiêu
- Phân tích chất lượng dữ liệu hiện tại
- Kiểm tra NULL values, duplicates, data types
- Phát hiện các vấn đề cần làm sạch
- Đánh giá tổng thể chất lượng dữ liệu


In [16]:
# Phân tích chất lượng dữ liệu
print("=== DATA QUALITY ANALYSIS ===")

# 1. Null ratio analysis
print("\n1. NULL RATIO ANALYSIS")
null_ratio = customers_df.isnull().mean().sort_values(ascending=False)
print("Tỷ lệ null theo cột:")
for col, ratio in null_ratio.items():
    print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")

# 2. Duplicate check
print(f"\n2. DUPLICATE CHECK")
dup_count = customers_df["customer_id"].duplicated().sum()
print(f"Số lượng customer_id trùng lặp: {dup_count}")

# 3. Value analysis cho các trường quan trọng
print(f"\n3. VALUE ANALYSIS")

# Gender analysis
print("\n--- GENDER ANALYSIS ---")
if 'gender' in customers_df.columns:
    gender_counts = customers_df['gender'].value_counts(dropna=False)
    print(gender_counts)

# Boolean fields analysis
boolean_fields = ['is_block', 'is_discount_by_level', 'active_levera_pay']
for field in boolean_fields:
    if field in customers_df.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = customers_df[field].value_counts(dropna=False)
        print(value_counts)

# Numeric fields analysis
numeric_fields = ['order_count', 'purchased_amount', 'reward_point', 'current_debts']
for field in numeric_fields:
    if field in customers_df.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        stats = customers_df[field].describe()
        print(f"Count: {stats['count']}")
        print(f"Mean: {stats['mean']:.2f}")
        print(f"Min: {stats['min']}")
        print(f"Max: {stats['max']}")
        print(f"Null count: {customers_df[field].isnull().sum()}")

# 4. Overall quality summary
print(f"\n4. OVERALL QUALITY SUMMARY")
total_columns = len(customers_df.columns)
high_null_columns = len(null_ratio[null_ratio > 0.7])
print(f"Total columns: {total_columns}")
print(f"Columns with >70% null: {high_null_columns}")
print(f"Duplicate customers: {dup_count}")
print(f"Data quality score: {((total_columns - high_null_columns) / total_columns * 100):.1f}%")


=== DATA QUALITY ANALYSIS ===

1. NULL RATIO ANALYSIS
Tỷ lệ null theo cột:
  last_order_at: 0.261 (26.1%)
  conversation_link: 0.212 (21.2%)
  gender: 0.149 (14.9%)
  address_commune_id: 0.101 (10.1%)
  address_district_id: 0.101 (10.1%)
  address_province_id: 0.101 (10.1%)
  address_full: 0.101 (10.1%)
  fb_id: 0.004 (0.4%)
  phone: 0.002 (0.2%)
  order_sources: 0.001 (0.1%)
  name: 0.000 (0.0%)
  purchased_amount: 0.000 (0.0%)
  returned_order_count: 0.000 (0.0%)
  succeed_order_count: 0.000 (0.0%)
  order_count: 0.000 (0.0%)
  shop_id: 0.000 (0.0%)
  id: 0.000 (0.0%)
  customer_id: 0.000 (0.0%)
  inserted_at: 0.000 (0.0%)
  count_referrals: 0.000 (0.0%)
  current_debts: 0.000 (0.0%)
  reward_point: 0.000 (0.0%)
  active_levera_pay: 0.000 (0.0%)
  is_discount_by_level: 0.000 (0.0%)
  is_block: 0.000 (0.0%)
  updated_at: 0.000 (0.0%)
  referral_code: 0.000 (0.0%)

2. DUPLICATE CHECK
Số lượng customer_id trùng lặp: 0

3. VALUE ANALYSIS

--- GENDER ANALYSIS ---
gender
F       19486
M   

## Bước 4: Xử lý Gender Field

### Mục tiêu
- Chuẩn hóa gender field
- Xử lý null values và giá trị không hợp lệ
- Đảm bảo consistency cho phân tích


In [17]:
# Xử lý Gender Field
print("=== GENDER FIELD CLEANING ===")

# Tạo bản copy để xử lý
customers_clean = customers_df.copy()

# Hiển thị trạng thái hiện tại
print("Trạng thái hiện tại:")
print(customers_clean['gender'].value_counts(dropna=False))

# Chuẩn hóa gender field
# Xử lý null values và chuẩn hóa
customers_clean['gender'] = customers_clean['gender'].fillna('Unknown')

# Chuẩn hóa các giá trị hiện có
gender_mapping = {
    'F': 'Nữ',
    'M': 'Nam', 
    'O': 'Unknown'
}

customers_clean['gender'] = customers_clean['gender'].map(gender_mapping).fillna('Unknown')

# Hiển thị kết quả
print("Kết quả sau khi chuẩn hóa:")
print(customers_clean['gender'].value_counts(dropna=False))

print(f"\nTổng số records: {len(customers_clean)}")
print(f"Null values: {customers_clean['gender'].isnull().sum()}")


=== GENDER FIELD CLEANING ===
Trạng thái hiện tại:
gender
F       19486
M       11239
None     5361
O           4
Name: count, dtype: int64
Kết quả sau khi chuẩn hóa:
gender
Nữ         19486
Nam        11239
Unknown     5365
Name: count, dtype: int64

Tổng số records: 36090
Null values: 0


## Bước 5: Xử lý Purchased Amount

### Mục tiêu
- Xử lý giá trị âm trong purchased_amount
- Đảm bảo tính hợp lệ của dữ liệu tài chính
- Chuẩn hóa giá trị cho phân tích


In [18]:
# Xử lý Purchased Amount
print("=== PURCHASED AMOUNT CLEANING ===")

# Hiển thị trạng thái hiện tại
print("Trạng thái hiện tại:")
print(f"Min: {customers_clean['purchased_amount'].min()}")
print(f"Max: {customers_clean['purchased_amount'].max()}")
print(f"Mean: {customers_clean['purchased_amount'].mean():.2f}")

# Kiểm tra giá trị âm
negative_count = (customers_clean['purchased_amount'] < 0).sum()
print(f"Số lượng giá trị âm: {negative_count}")

if negative_count > 0:
    print("Các giá trị âm:")
    negative_values = customers_clean[customers_clean['purchased_amount'] < 0]['purchased_amount'].value_counts()
    print(negative_values)

# Xử lý giá trị âm - lấy trị tuyệt đối
print("\nXử lý giá trị âm...")
customers_clean['purchased_amount'] = customers_clean['purchased_amount'].abs()

# Hiển thị kết quả
print("Kết quả sau khi xử lý:")
print(f"Min: {customers_clean['purchased_amount'].min()}")
print(f"Max: {customers_clean['purchased_amount'].max()}")
print(f"Mean: {customers_clean['purchased_amount'].mean():.2f}")

# Kiểm tra lại giá trị âm
negative_count_after = (customers_clean['purchased_amount'] < 0).sum()
print(f"Số lượng giá trị âm sau xử lý: {negative_count_after}")


=== PURCHASED AMOUNT CLEANING ===
Trạng thái hiện tại:
Min: -1788000.0
Max: 6199000.0
Mean: 222192.57
Số lượng giá trị âm: 59
Các giá trị âm:
purchased_amount
-50000.0      4
-40000.0      3
-100000.0     3
-240000.0     3
-300000.0     3
-30000.0      3
-49000.0      3
-58000.0      3
-340000.0     2
-29000.0      2
-78000.0      2
-80000.0      2
-21000.0      2
-140000.0     2
-110000.0     2
-179000.0     1
-180000.0     1
-98000.0      1
-39000.0      1
-108000.0     1
-190000.0     1
-795000.0     1
-150000.0     1
-1788000.0    1
-90000.0      1
-8000.0       1
-60000.0      1
-620000.0     1
-120000.0     1
-116000.0     1
-88000.0      1
-41000.0      1
-162000.0     1
-70000.0      1
-157000.0     1
Name: count, dtype: int64

Xử lý giá trị âm...
Kết quả sau khi xử lý:
Min: 0.0
Max: 6199000.0
Mean: 222708.28
Số lượng giá trị âm sau xử lý: 0


## Bước 6: Chuẩn hóa Name Field

### Mục tiêu
- Chuẩn hóa tên khách hàng
- Loại bỏ khoảng trắng thừa
- Áp dụng title case
- Xử lý tên trống hoặc không hợp lệ


In [19]:
# Chuẩn hóa Name Field
print("=== NAME FIELD CLEANING ===")

# Hiển thị trạng thái hiện tại
print("Trạng thái hiện tại:")
print(f"Null values: {customers_clean['name'].isnull().sum()}")
print(f"Empty strings: {(customers_clean['name'] == '').sum()}")

# Sample data trước khi xử lý
print("\nSample data trước khi xử lý:")
print(customers_clean['name'].head(10).tolist())

# Chuẩn hóa name field
print("\nChuẩn hóa name field...")

# Loại bỏ khoảng trắng thừa và áp dụng title case
customers_clean['name'] = customers_clean['name'].str.strip().str.title()

# Xử lý tên trống hoặc không hợp lệ
customers_clean['name'] = customers_clean['name'].replace('', 'Unknown')
customers_clean['name'] = customers_clean['name'].fillna('Unknown')

# Hiển thị kết quả
print("Kết quả sau khi chuẩn hóa:")
print(f"Null values: {customers_clean['name'].isnull().sum()}")
print(f"Empty strings: {(customers_clean['name'] == '').sum()}")

print("\nSample data sau khi xử lý:")
print(customers_clean['name'].head(10).tolist())

# Kiểm tra tên "Unknown"
unknown_count = (customers_clean['name'] == 'Unknown').sum()
print(f"Số lượng tên 'Unknown': {unknown_count}")


=== NAME FIELD CLEANING ===
Trạng thái hiện tại:
Null values: 7
Empty strings: 0

Sample data trước khi xử lý:
['Thinh Bui', 'Truong Minh', 'Quach Quach', 'Dung Vu', 'Phạm Mụi', 'Trần Thu', 'Nguyễn Thế Kiên', 'Toai Vu', 'Du Pham', 'Lê Hữu Phước']

Chuẩn hóa name field...
Kết quả sau khi chuẩn hóa:
Null values: 0
Empty strings: 0

Sample data sau khi xử lý:
['Thinh Bui', 'Truong Minh', 'Quach Quach', 'Dung Vu', 'Phạm Mụi', 'Trần Thu', 'Nguyễn Thế Kiên', 'Toai Vu', 'Du Pham', 'Lê Hữu Phước']
Số lượng tên 'Unknown': 7


## Bước 7: Chuẩn hóa Phone Field

### Mục tiêu
- Chuẩn hóa số điện thoại
- Loại bỏ ký tự đặc biệt
- Chỉ giữ lại số
- Xử lý phone trống hoặc không hợp lệ


In [20]:
# Chuẩn hóa Phone Field
print("=== PHONE FIELD CLEANING ===")

# Hiển thị trạng thái hiện tại
print("Trạng thái hiện tại:")
print(f"Null values: {customers_clean['phone'].isnull().sum()}")
print(f"Empty strings: {(customers_clean['phone'] == '').sum()}")

# Sample data trước khi xử lý
print("\nSample data trước khi xử lý:")
print(customers_clean['phone'].head(10).tolist())

# Chuẩn hóa phone field
print("\nChuẩn hóa phone field...")

# Loại bỏ ký tự đặc biệt, chỉ giữ lại số
customers_clean['phone'] = customers_clean['phone'].str.replace(r'[^\d]', '', regex=True)

# Xử lý phone trống hoặc không hợp lệ
customers_clean['phone'] = customers_clean['phone'].replace('', None)
customers_clean['phone'] = customers_clean['phone'].fillna('Unknown')

# Hiển thị kết quả
print("Kết quả sau khi chuẩn hóa:")
print(f"Null values: {customers_clean['phone'].isnull().sum()}")
print(f"Empty strings: {(customers_clean['phone'] == '').sum()}")

print("\nSample data sau khi xử lý:")
print(customers_clean['phone'].head(10).tolist())

# Kiểm tra phone "Unknown"
unknown_phone_count = (customers_clean['phone'] == 'Unknown').sum()
print(f"Số lượng phone 'Unknown': {unknown_phone_count}")

# Kiểm tra độ dài phone number
phone_lengths = customers_clean['phone'].str.len().value_counts().sort_index()
print(f"\nPhân bố độ dài phone number:")
print(phone_lengths)


=== PHONE FIELD CLEANING ===
Trạng thái hiện tại:
Null values: 90
Empty strings: 0

Sample data trước khi xử lý:
['0903693389', '0907809070', '0986533988', '0977752936', '0774443833', '0368838710', '0972868119', '0395028188', '0907007799', '0976282162']

Chuẩn hóa phone field...
Kết quả sau khi chuẩn hóa:
Null values: 0
Empty strings: 0

Sample data sau khi xử lý:
['0903693389', '0907809070', '0986533988', '0977752936', '0774443833', '0368838710', '0972868119', '0395028188', '0907007799', '0976282162']
Số lượng phone 'Unknown': 90

Phân bố độ dài phone number:
phone
7        90
9         1
10    34987
11      571
18        1
20      313
21      116
30        8
31        3
Name: count, dtype: int64


## Bước 8: Chuẩn hóa Address Fields

### Mục tiêu
- Chuẩn hóa các trường địa chỉ
- Xử lý null values trong address
- Chuẩn hóa format địa chỉ


In [21]:
# Chuẩn hóa Address Fields
print("=== ADDRESS FIELDS CLEANING ===")

# Kiểm tra các trường address
address_fields = ['address_full', 'address_province_id', 'address_district_id', 'address_commune_id']

print("Trạng thái hiện tại của address fields:")
for field in address_fields:
    if field in customers_clean.columns:
        null_count = customers_clean[field].isnull().sum()
        null_pct = (null_count / len(customers_clean)) * 100
        print(f"  {field}: {null_count} null ({null_pct:.1f}%)")

# Chuẩn hóa address_full
if 'address_full' in customers_clean.columns:
    print("\nChuẩn hóa address_full...")
    # Loại bỏ khoảng trắng thừa
    customers_clean['address_full'] = customers_clean['address_full'].str.strip()
    # Xử lý null values
    customers_clean['address_full'] = customers_clean['address_full'].fillna('Unknown')

# Chuẩn hóa address IDs
for field in ['address_province_id', 'address_district_id', 'address_commune_id']:
    if field in customers_clean.columns:
        print(f"\nChuẩn hóa {field}...")
        # Xử lý null values
        customers_clean[field] = customers_clean[field].fillna('Unknown')

# Hiển thị kết quả
print("\nKết quả sau khi chuẩn hóa:")
for field in address_fields:
    if field in customers_clean.columns:
        null_count = customers_clean[field].isnull().sum()
        unknown_count = (customers_clean[field] == 'Unknown').sum()
        print(f"  {field}: {null_count} null, {unknown_count} Unknown")

# Sample data
print("\nSample address data:")
sample_address = customers_clean[['customer_id', 'name', 'address_full', 'address_province_id']].head(5)
print(sample_address)


=== ADDRESS FIELDS CLEANING ===
Trạng thái hiện tại của address fields:
  address_full: 3631 null (10.1%)
  address_province_id: 3641 null (10.1%)
  address_district_id: 3646 null (10.1%)
  address_commune_id: 3649 null (10.1%)

Chuẩn hóa address_full...

Chuẩn hóa address_province_id...

Chuẩn hóa address_district_id...

Chuẩn hóa address_commune_id...

Kết quả sau khi chuẩn hóa:
  address_full: 0 null, 3631 Unknown
  address_province_id: 0 null, 3641 Unknown
  address_district_id: 0 null, 3646 Unknown
  address_commune_id: 0 null, 3649 Unknown

Sample address data:
                            customer_id         name address_full  \
0  c7a583d6-392d-4308-a02b-67c001ee03db    Thinh Bui      Unknown   
1  590ba7de-e4fe-43b3-8013-27fc69937edf  Truong Minh      Unknown   
2  a9e62388-57c2-4834-a87c-7c1025aea92d  Quach Quach      Unknown   
3  002810c4-5e96-4882-b85a-a85122e47108      Dung Vu      Unknown   
4  cd25d874-d782-40d1-8561-5507bbd6114d     Phạm Mụi      Unknown   

  address_p

## Bước 9: Chuẩn hóa Datetime Fields

### Mục tiêu
- Chuẩn hóa các trường datetime
- Đảm bảo format datetime đúng
- Xử lý null values trong datetime


In [22]:
# Chuẩn hóa Datetime Fields
print("=== DATETIME FIELDS CLEANING ===")

# Kiểm tra các trường datetime
datetime_fields = ['inserted_at', 'updated_at', 'last_order_at']

print("Trạng thái hiện tại của datetime fields:")
for field in datetime_fields:
    if field in customers_clean.columns:
        null_count = customers_clean[field].isnull().sum()
        null_pct = (null_count / len(customers_clean)) * 100
        print(f"  {field}: {null_count} null ({null_pct:.1f}%)")

# Chuẩn hóa datetime fields
for field in datetime_fields:
    if field in customers_clean.columns:
        print(f"\nChuẩn hóa {field}...")
        # Convert to datetime
        customers_clean[field] = pd.to_datetime(customers_clean[field], errors='coerce')
        
        # Hiển thị thông tin sau khi convert
        null_count = customers_clean[field].isnull().sum()
        print(f"  {field}: {null_count} null sau khi convert")

# Hiển thị kết quả
print("\nKết quả sau khi chuẩn hóa:")
for field in datetime_fields:
    if field in customers_clean.columns:
        null_count = customers_clean[field].isnull().sum()
        print(f"  {field}: {null_count} null")
        
        # Sample data
        if null_count < len(customers_clean):
            sample_dates = customers_clean[field].dropna().head(3)
            print(f"    Sample dates: {sample_dates.tolist()}")

# Kiểm tra data types
print("\nData types sau khi chuẩn hóa:")
for field in datetime_fields:
    if field in customers_clean.columns:
        print(f"  {field}: {customers_clean[field].dtype}")


=== DATETIME FIELDS CLEANING ===
Trạng thái hiện tại của datetime fields:
  inserted_at: 0 null (0.0%)
  updated_at: 0 null (0.0%)
  last_order_at: 9414 null (26.1%)

Chuẩn hóa inserted_at...
  inserted_at: 0 null sau khi convert

Chuẩn hóa updated_at...
  updated_at: 0 null sau khi convert

Chuẩn hóa last_order_at...
  last_order_at: 9414 null sau khi convert

Kết quả sau khi chuẩn hóa:
  inserted_at: 0 null
    Sample dates: [Timestamp('2025-08-16 02:19:59'), Timestamp('2025-08-15 12:56:50'), Timestamp('2025-08-15 12:45:17')]
  updated_at: 0 null
    Sample dates: [Timestamp('2025-08-16 02:19:59'), Timestamp('2025-08-15 12:56:50'), Timestamp('2025-08-15 12:45:17')]
  last_order_at: 9414 null
    Sample dates: [Timestamp('2025-04-16 05:47:31'), Timestamp('2025-04-15 02:22:05'), Timestamp('2025-04-16 09:45:24')]

Data types sau khi chuẩn hóa:
  inserted_at: datetime64[ns]
  updated_at: datetime64[ns]
  last_order_at: datetime64[ns]


## Bước 10: Tổng kết và Validation

### Mục tiêu
- Kiểm tra tổng thể dữ liệu sau khi làm sạch
- So sánh trước và sau khi xử lý
- Chuẩn bị cho việc load vào Gold database


In [23]:
# Tổng kết và Validation
print("=== FINAL VALIDATION ===")

# So sánh trước và sau khi xử lý
print("So sánh trước và sau khi xử lý:")
print(f"Original shape: {customers_df.shape}")
print(f"Cleaned shape: {customers_clean.shape}")

# Kiểm tra null values tổng thể
print("\nNull values analysis:")
original_nulls = customers_df.isnull().sum().sum()
cleaned_nulls = customers_clean.isnull().sum().sum()
print(f"Original total nulls: {original_nulls}")
print(f"Cleaned total nulls: {cleaned_nulls}")

# Kiểm tra data types
print("\nData types comparison:")
print("Original data types:")
print(customers_df.dtypes.value_counts())
print("\nCleaned data types:")
print(customers_clean.dtypes.value_counts())

# Kiểm tra các trường quan trọng
print("\nKey fields validation:")
key_fields = ['customer_id', 'name', 'gender', 'phone', 'purchased_amount']
for field in key_fields:
    if field in customers_clean.columns:
        null_count = customers_clean[field].isnull().sum()
        unique_count = customers_clean[field].nunique()
        print(f"  {field}: {null_count} null, {unique_count} unique values")

# Sample cleaned data
print("\nSample cleaned data:")
sample_cleaned = customers_clean[['customer_id', 'name', 'gender', 'phone', 'purchased_amount', 'order_count']].head(5)
print(sample_cleaned)

# Memory usage
print(f"\nMemory usage:")
print(f"Original: {customers_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"Cleaned: {customers_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

print(f"\nCleaning completed successfully!")
print(f"Ready for loading to Gold database")


=== FINAL VALIDATION ===
So sánh trước và sau khi xử lý:
Original shape: (36090, 27)
Cleaned shape: (36090, 27)

Null values analysis:
Original total nulls: 37252
Cleaned total nulls: 17227

Data types comparison:
Original data types:
object            13
int64             10
datetime64[ns]     3
float64            1
Name: count, dtype: int64

Cleaned data types:
object            13
int64             10
datetime64[ns]     3
float64            1
Name: count, dtype: int64

Key fields validation:
  customer_id: 0 null, 36090 unique values
  name: 0 null, 25191 unique values
  gender: 0 null, 3 unique values
  phone: 0 null, 35717 unique values
  purchased_amount: 0 null, 526 unique values

Sample cleaned data:
                            customer_id         name gender       phone  \
0  c7a583d6-392d-4308-a02b-67c001ee03db    Thinh Bui    Nam  0903693389   
1  590ba7de-e4fe-43b3-8013-27fc69937edf  Truong Minh    Nam  0907809070   
2  a9e62388-57c2-4834-a87c-7c1025aea92d  Quach Quach    N

In [25]:
# Xử lý Primary Key
print("=== PRIMARY KEY PROCESSING ===")

# Hiển thị trạng thái hiện tại
print("Trạng thái hiện tại:")
print(f"Columns: {list(customers_clean.columns)}")
print(f"customer_id null: {customers_clean['customer_id'].isnull().sum()}")
print(f"id null: {customers_clean['id'].isnull().sum()}")

# Xóa cột customer_id cũ
print("\nXóa cột customer_id cũ...")
customers_clean = customers_clean.drop(columns=['customer_id'])

# Đổi tên cột id thành customer_id
print("Đổi tên cột id thành customer_id...")
customers_clean = customers_clean.rename(columns={'id': 'customer_id'})

# Hiển thị kết quả
print("Kết quả sau khi xử lý:")
print(f"Columns: {list(customers_clean.columns)}")
print(f"customer_id null: {customers_clean['customer_id'].isnull().sum()}")
print(f"customer_id unique: {customers_clean['customer_id'].nunique()}")

# Sample data
print("\nSample data với customer_id mới:")
sample_data = customers_clean[['customer_id', 'name', 'gender', 'phone']].head(3)
print(sample_data)

=== PRIMARY KEY PROCESSING ===
Trạng thái hiện tại:
Columns: ['customer_id', 'id', 'name', 'shop_id', 'gender', 'phone', 'order_count', 'succeed_order_count', 'returned_order_count', 'purchased_amount', 'last_order_at', 'inserted_at', 'updated_at', 'is_block', 'is_discount_by_level', 'active_levera_pay', 'reward_point', 'current_debts', 'count_referrals', 'referral_code', 'fb_id', 'order_sources', 'conversation_link', 'address_province_id', 'address_district_id', 'address_commune_id', 'address_full']
customer_id null: 0
id null: 0

Xóa cột customer_id cũ...
Đổi tên cột id thành customer_id...
Kết quả sau khi xử lý:
Columns: ['customer_id', 'name', 'shop_id', 'gender', 'phone', 'order_count', 'succeed_order_count', 'returned_order_count', 'purchased_amount', 'last_order_at', 'inserted_at', 'updated_at', 'is_block', 'is_discount_by_level', 'active_levera_pay', 'reward_point', 'current_debts', 'count_referrals', 'referral_code', 'fb_id', 'order_sources', 'conversation_link', 'address_prov

In [26]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, Float, DateTime, Text, VARCHAR, Boolean

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - VARCHAR(100)
id_fields = ["customer_id", "fb_id"]
for field in id_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(100)

# Name fields - VARCHAR(255)  
name_fields = ["name", "referral_code"]
for field in name_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(255)

# Text fields - Text()
text_fields = ["phone", "conversation_link", "address_full", "order_sources"]
for field in text_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Text()

# Numeric fields
if "shop_id" in customers_clean.columns:
    dtype_mapping["shop_id"] = BigInteger()

# Integer fields
integer_fields = ["order_count", "succeed_order_count", "returned_order_count", 
                  "reward_point", "current_debts", "count_referrals"]
for field in integer_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Integer()

# Float fields
float_fields = ["purchased_amount"]
for field in float_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["inserted_at", "updated_at", "last_order_at"]
for field in datetime_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = DateTime()

# Boolean fields
boolean_fields = ["is_block", "is_discount_by_level", "active_levera_pay"]
for field in boolean_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = Boolean()

# String fields (default)
string_fields = ["gender", "address_province_id", "address_district_id", 
                "address_commune_id"]
for field in string_fields:
    if field in customers_clean.columns:
        dtype_mapping[field] = VARCHAR(50)

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_customers"

try:
    customers_clean.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {customers_clean.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT customer_id, name, gender, phone, purchased_amount FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:")
    print(sample_db)
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 26 columns
Đã load 36090 records vào Gold: dim_customers
Verification: 36090 records trong database
Schema: 26 columns với explicit typing

Sample data từ Gold database:
                            customer_id         name gender       phone  \
0  e906c5c7-ea19-42e5-a971-57cf09c94417    Thinh Bui    Nam  0903693389   
1  6c5b40d2-ae42-4e13-8019-f3f9127d75a0  Truong Minh    Nam  0907809070   
2  765c531b-743f-4f30-b6b0-0d6f3579630f  Quach Quach    Nam  0986533988   
3  f169b9a8-9a87-4c74-a296-ba9b505c63a6      Dung Vu    Nam  0977752936   
4  4b7b436f-93ff-49ee-89e5-157eb85ff1bb     Phạm Mụi    Nam  0774443833   

   purchased_amount  
0               0.0  
1               0.0  
2               0.0  
3               0.0  
4               0.0  


In [28]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "customer_id": "Mã định danh khách hàng duy nhất (khóa chính)",
        "name": "Tên đầy đủ của khách hàng",
        "shop_id": "Mã định danh cửa hàng (khóa ngoại đến dim_shops)",
        
        # Thông tin cá nhân
        "gender": "Giới tính khách hàng (Nam/Nữ/Unknown)",
        "phone": "Số điện thoại liên hệ của khách hàng",
        
        # Hành vi mua hàng
        "order_count": "Tổng số đơn hàng đã đặt",
        "succeed_order_count": "Số đơn hàng thành công",
        "returned_order_count": "Số đơn hàng đã trả",
        "purchased_amount": "Tổng số tiền đã mua (VND)",
        "last_order_at": "Ngày đặt đơn hàng cuối cùng",
        
        # Thời gian
        "inserted_at": "Thời gian tạo bản ghi khách hàng",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng",
        
        # Trạng thái
        "is_block": "Trạng thái bị khóa tài khoản (true/false)",
        "is_discount_by_level": "Có được giảm giá theo cấp độ (true/false)",
        "active_levera_pay": "Có sử dụng thanh toán Levera (true/false)",
        
        # Tài chính & Loyalty
        "reward_point": "Điểm thưởng hiện tại",
        "current_debts": "Số nợ hiện tại",
        "count_referrals": "Số lần giới thiệu thành công",
        
        # Marketing & Tracking
        "referral_code": "Mã giới thiệu của khách hàng",
        "fb_id": "ID Facebook để theo dõi mạng xã hội",
        "order_sources": "Nguồn đơn hàng (kênh marketing)",
        "conversation_link": "Link hội thoại với khách hàng",
        
        # Địa chỉ
        "address_province_id": "Mã tỉnh/thành phố",
        "address_district_id": "Mã quận/huyện",
        "address_commune_id": "Mã phường/xã",
        "address_full": "Địa chỉ đầy đủ"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in customers_clean.columns:
    col_info = {
        "table_name": "dim_customers",
        "column_name": col,
        "dtype": str(customers_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": customers_clean[col].isnull().sum(),
        "null_pct": round(customers_clean[col].isnull().mean() * 100, 2),
        "unique_count": customers_clean[col].nunique(),
        "sample_values": str(customers_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_customers")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)

=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 26 columns

=== DATA DICTIONARY ===
       table_name           column_name           dtype      sql_type  \
0   dim_customers           customer_id          object  VARCHAR(100)   
1   dim_customers                  name          object  VARCHAR(255)   
2   dim_customers               shop_id           int64        BIGINT   
3   dim_customers                gender          object   VARCHAR(50)   
4   dim_customers                 phone          object          TEXT   
5   dim_customers           order_count           int64       INTEGER   
6   dim_customers   succeed_order_count           int64       INTEGER   
7   dim_customers  returned_order_count           int64       INTEGER   
8   dim_customers      purchased_amount         float64         FLOAT   
9   dim_customers         last_order_at  datetime64[ns]      DATETIME   
10  dim_customers           inserted_at  datetime64[ns]      DATETIME   
11  dim_customers   